# Automated finite difference operators from symbolic equations

This is the first part. It will give an overview...

In [ ]:
from devito import *
from sympy import init_printing, symbols, solve
init_printing(use_latex=True)

**[TODO: Introduce basic concepts]**

A `Function` object does two things:
1. It behaves like a `sympy.Function` symbol
2. It carries around user data

In [ ]:
grid = Grid(shape=(5, 6), extent=(1., 1.))
f = Function(name='f', grid=grid)

## Symbolic behaviour of Function objects

Let's have a look at the symbolic behaviour first by taking a derivative in `x`. For this we first create a derivative symbol and then we evaluate it as a finite difference expression.

In [ ]:
f

In [ ]:
f.diff(x)

Using SymPy we can now discretize this derivative symbol using the finite difference method. First, we let SymPy create a generic central difference expression.

In [ ]:
f.diff(x).as_finite_difference()

Next we can tell SymPy to insert the symbol $h_x$ to represent the spacing of our grid in the $x$ dimension. SymPy will still use a central differencing scheme, where we ae required to know the values of $f(x, y)$ at distances $h_x / 2$.

In [ ]:
h_x = symbols('h_x')
f.diff(x).as_finite_difference(h_x)

To avoid values half-cell offsets we can force SymPy to use a forward differencing scheme by providing the explicit stencil points for which to discretize our derivative. This stencil expression now aligns with our computational grid.

In [ ]:
f.diff(x).as_finite_difference([x, x+h_x])

Typing out this expression every time I want to take a simple derivative is somewhat tedious. Luckily, Devito symbols provide a set of shorthand notations to generate the most common derivatives automatically. The first derivative in x, for example, is simply: 

In [ ]:
f.dx

## Time-dependent functions and data

**[TODO: Explain TimeFunction and .data]**

In [ ]:
# TODO: Show u.dt and u.data with "hat" initial data

## A simple operator: Linear convenction

**[TODO: Explain the role of Operator and how codegen works]**

**Note:** The following example is derived from [step 5](http://nbviewer.ipython.org/github/barbagroup/CFDPython/blob/master/lessons/07_Step_5.ipynb) of the tutorials in the excellent tutorial series [CFD Python: 12 steps to Navier-Stokes](http://lorenabarba.com/blog/cfd-python-12-steps-to-navier-stokes/).

In [ ]:
# TODO: Build the operator by letting SymPy solve for f(t + s, x, y)
# Then build operator and run it

## Exercise 1: A coupled system

The solution of the previous example showed a lot of numerical diffusion. This is somewhat expected as the discretization we have chosen is not very good at capturing sharp gradients, like the edge of our "hat". A better formulation of the equation is given by the coupled system of equations:

**[TODO: Spell out coupled formulation]**

To implement a coupled set of equations, all we have to do is create update expressions for both state variables and handing both expressions to the `Operator`. This is simply done by concatenating them in a Python list.

In [ ]:
# Hint: We need to create two expressions, one to update u(t+s, ...) and one for v(t+s, ...).
# Both need to depend on the state of variables u(t, ...) and v(t, ...).

<button data-toggle="collapse" data-target="#sol1" class='btn btn-primary'>Solution</button>
<div id="sol1" class="collapse">
```python
# TODO: Reference solution
```

## Second derivatives and high-order stencils

For the above example all we had to do was combine some first derivatives. However, lots of common scientific problems require second derivative, most notably any PDE including diffusion. To generate second order derivatives we need to give the `devito.Function` object another piece of information: the desired discretization of the stencils. 

First, let's do a simple second derivative in %x%, for which we need to give %u% at least a `space_order` of `2`. The shorthand for the second derivative is then `u.dx2`. 

In [ ]:
u = TimeFunction(name='u', grid=grid, space_order=2)
u.dx2

We can arbitrarily drive the discretization order up if require higher order stencils. 

In [ ]:
u = TimeFunction(name='u', grid=grid, space_order=4)
u.dx2

To implement a diffusion operator, we need to take the Laplacian $\nabla^2 u$, which is simply the second derivative in all space dimensions. For this, Devito also provides a shorthand expression, which means I do not have to hardcode the problem dimension (2D or 3D) in my code. I simply give another shape to the grid and let it figure our the Laplacian like this:

In [ ]:
grid_3d = Grid(shape=(5, 6, 7), extent=(1., 1., 1.))
u = TimeFunction(name='u', grid=grid_3d, space_order=2)
u.laplace

Just for kicks (and to show off the power of SymPy), let's see what the 12th-order Laplacian of a three dimensional time-dependent function looks like.

In [ ]:
grid_3d = Grid(shape=(5, 6, 7), extent=(1., 1., 1.))
u = TimeFunction(name='u', grid=grid_3d, space_order=12)
u.laplace

## Exercise 2: Making a wave

In the final exercise of the introduction we will implement a wave equation operator and visualize a wave propagating from a single source point.

**[TODO: Problem spec with equation]**

In [ ]:
# Please have a go and try to implement the operator.
# Hint: Your final line should define `op = Operator(...)

<button data-toggle="collapse" data-target="#sol1" class='btn btn-primary'>Solution</button>
<div id="sol1" class="collapse">
```python
grid = Grid(shape=(5, 6))
u = TimeData(name='u', space_order=2, time_order=2)
m = DenseData(name='m')
eqn = Eq(m * u.dt2 - u.laplace)
stencil = solve(eqn, u.forward)[0]
op = Operator(stencil)
```

In [ ]:
# TODO: Run operator with animation